In [ ]:
WEIGHT_METRICS = False

In [1]:
import pandas as pd
df = pd.read_csv("./keyterm_scored.csv")
del df['idx']
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score_normalized = df.abstraction_score_normalized.apply(eval)
df.reading_ease_normalized = df.reading_ease_normalized.apply(eval)
df.grading_level_normalized = df.grading_level_normalized.apply(eval) 
df.sgrank_normalized = df.sgrank_normalized.apply(eval)
df.sample(frac = 1).head(20)

,prompt,response,clauses_text_final,voice,abstraction_score_normalized,reading_ease_normalized,grading_level_normalized,sgrank_normalized
125,A good boss,"triggers no response in me at the moment, albe...","[triggers no response in me at the moment, alb...","[A_def, P_bevb_x, P_bevb_x, A_def, A_def, A_de...","[0.56, 1.0, 0.56, 1.0, 1.0, 1.0, 0.56, 1.0]","[0.11, 0.05, 0.13, 0.13, 0.1, 0.38, 0.14, 0.1]","[0.15, 0.09, 0.13, 0.2, 0.12, 0.38, 0.17, 0.15]","[0.02, 0.03, 0.0, 0.25, 0.03, 0.0, 0.0, 0.02]"
372,Crime and delinquency could be halted if,...we don't make rules and laws that define ce...,"[we don t, make rules and laws, that define ce...","[P_bevb_x, A_def, A_def, A_def, A_def, A_def, ...","[0.56, 0.56, 1.0, 0.88, 0.56, 0.56, 0.56, 0.56...","[0.01, 0.01, 0.21, 0.0, 0.19, 0.08, 0.01, 0.16...","[0.02, 0.02, 0.25, 0.01, 0.27, 0.12, 0.03, 0.1...","[0.0, 0.03, 0.08, 0.0, 0.2, 0.02, 0.02, 0.53, ..."
78,When developing strategies&hellip;,"express their needs, their emotions, to be hon...","[their needs their emotions, to be honest, to ...","[Undefined, P_bevb_x, A_def, P_bevb_x, P_bevb_...","[0.56, 0.56, 1.0, 0.48, 0.56, 0.48, 1.0, 0.56,...","[0.07, 0.09, 0.16, 0.09, 0.14, 0.17, 0.5, 0.0,...","[0.09, 0.1, 0.18, 0.1, 0.18, 0.18, 0.51, 0.01,...","[0.07, 0.0, 0.1, 0.19, 0.19, 0.0, 0.0, 0.0, 0...."
327,When I get mad,I sometimes use big voice then I go somewhere ...,"[I sometimes use big voice, then I go somewher...","[A_def, P_get_x, P_bevb_x]","[1.0, 0.88, 1.0]","[0.06, 0.11, 0.06]","[0.08, 0.13, 0.08]","[0.56, 0.24, 0.0]"
219,The thing I like about myself is,The thing I like about myself is... I am willi...,"[The thing, I like about myself, is, I am will...","[Undefined, A_def, P_bevb_x, P_bevb_x, A_def, ...","[0.56, 0.88, 0.0, 0.88, 0.56, 0.56, 0.88, 1.0,...","[0.0, 0.07, 0.0, 0.11, 0.01, 0.2, 0.0, 0.1, 0....","[0.01, 0.09, 0.0, 0.15, 0.02, 0.21, 0.01, 0.15...","[0.05, 0.0, 0.0, 0.01, 0.05, 0.03, 0.0, 0.01, ..."
426,"These days, teaching","is a very demanding profession, and in my expe...","[a very demanding profession and, in my experi...","[Undefined, A_pron_x, P_bevb_x, A_def]","[0.56, 1.0, 0.56, 0.56]","[0.21, 0.34, 0.14, 0.2]","[0.23, 0.37, 0.17, 0.23]","[0.17, 0.25, 0.0, 0.06]"
457,What I like to do best is,play.,[play],[A_def],[0.0],[0.0],[0.0],[0.0]
358,A good child,"is what like good child does, they say things ...","[is, what like good child does they say things...","[P_bevb_x, P_bevb_x, A_def, A_def, P_bevb_x]","[0.0, 1.0, 1.0, 0.88, 0.56]","[0.0, 0.02, 0.01, 0.09, 0.06]","[0.0, 0.06, 0.03, 0.1, 0.08]","[0.0, 0.44, 0.12, 0.0, 0.0]"
352,Rules,are useful and sometimes not useful.,[are useful and sometimes not useful],[P_bevb_x],[0.56],[0.14],[0.17],[0.93]
182,My father,was a thread of eternity through the instant o...,[was a thread of eternity through the instant ...,"[P_bevb_x, P_bevb_x]","[0.56, 0.88]","[0.13, 0.05]","[0.17, 0.09]","[0.11, 0.1]"


In [6]:
df['idx_abstraction_score'] = df['abstraction_score_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_grading_level'] = df['grading_level_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_reading_ease'] = df['reading_ease_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)
df['idx_sgrank'] = df['sgrank_normalized'].apply(lambda arr: arr.index(max(arr)) if len(arr)>0 else -1)

df[['prompt', 'response', 'clauses_text_final', 'voice', 'idx_abstraction_score', 'idx_grading_level', 'idx_reading_ease', 'idx_sgrank']].sample(frac = 1).head()

,prompt,response,clauses_text_final,voice,idx_abstraction_score,idx_grading_level,idx_reading_ease,idx_sgrank
26,Education,"on a need to know basis, it will be there wait...","[to know basis, it will be there waiting]","[A_def, P_bevb_x]",0,0,0,0
144,When people are helpless,I want to help.,[I want to help],"[A_def, A_def]",0,0,0,0
376,If I can\'t get what I want,I'm okay,[want],[A_def],0,0,0,0
249,Rules,would be more widely adopted and used if they ...,"[would be, more widely adopted and, used, if t...","[P_bevb_x, A_def, A_def, P_bevb_x]",3,3,3,3
367,What I like to do best is,read,[read],[A_def],0,0,0,0


In [3]:
df['voice_abstraction'] = df.apply(lambda row : row['voice'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else "Indeterminate", axis = 1)
df['voice_grading_level'] = df.apply(lambda row : row['voice'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else "Indeterminate", axis = 1)
df['voice_reading_ease'] = df.apply(lambda row : row['voice'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else "Indeterminate", axis = 1)
df['voice_sgrank'] = df.apply(lambda row : row['voice'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else "Indeterminate", axis = 1)

df['final_abstraction'] = df.apply(lambda row : row['abstraction_score_normalized'][row['idx_abstraction_score']] if row['idx_abstraction_score'] >=0 else -1, axis = 1)
df['final_grading_level'] = df.apply(lambda row : row['grading_level_normalized'][row['idx_grading_level']] if row['idx_grading_level'] >=0 else -1, axis = 1)
df['final_reading_ease'] = df.apply(lambda row : row['reading_ease_normalized'][row['idx_reading_ease']] if row['idx_reading_ease'] >=0 else -1, axis = 1)
df['final_sgrank'] = df.apply(lambda row : row['sgrank_normalized'][row['idx_sgrank']] if row['idx_sgrank'] >=0 else -1, axis = 1)

df[['prompt', 'response', 'clauses_text_final', 'voice', 'final_abstraction', 'voice_abstraction', 'final_grading_level', 'voice_grading_level', 'final_reading_ease', 'voice_reading_ease', 'final_sgrank', 'voice_sgrank']].head()
#df.iloc[342]

,prompt,response,clauses_text_final,voice,final_abstraction,voice_abstraction,final_grading_level,voice_grading_level,final_reading_ease,voice_reading_ease,final_sgrank,voice_sgrank
0,The past,"Winds through us, both from our lives and cult...",[Winds through us both from our lives and cult...,"[A_pron_x, A_pron_x, P_bevb_x, P_bevb_x, P_bev...",1.00,A_pron_x,0.42,A_pron_x,0.26,A_pron_x,0.12,P_bevb_x
1,At times I worry about,Insufficient care and attention for his egocen...,[Insufficient care and attention for his egoce...,[A_pron_x],1.00,A_pron_x,0.49,A_pron_x,0.31,A_pron_x,0.08,A_pron_x
2,What gets me into trouble is,not considering others possibilities.,[not considering others possibilities],[A_def],1.00,A_def,0.52,A_def,0.51,A_def,0.00,A_def
3,When people are helpless,They often don&#039;t know it so they flak aro...,"[They often don, t know it, so they flak aroun...","[P_bevb_x, A_def, A_def, A_def]",0.56,P_bevb_x,0.12,A_def,0.10,A_def,0.00,P_bevb_x
4,Privacy,is a sense of hiding from others that which yo...,"[is a sense of, hiding from others that, which...","[P_bevb_x, A_def, P_bevb_x, P_bevb_x, A_def, A...",1.00,P_bevb_x,0.21,A_def,0.20,A_def,0.65,P_bevb_x


In [7]:
df['sentinel'] = "Indeterminate"

abstraction_weight = 1/df['final_abstraction'].mean() if WEIGHT_METRICS else 1
grading_level_weight = 1/df['final_grading_level'].mean() if WEIGHT_METRICS else 1
reading_ease_weight = 1/df['final_reading_ease'].mean() if WEIGHT_METRICS else 1
sgrank_weight = 1/df['final_sgrank'].mean() if WEIGHT_METRICS else 1

cdict = {-1 : 'sentinel', 0 : 'voice_abstraction', 1 : 'voice_grading_level', 2 : 'voice_reading_ease', 3 : 'voice_sgrank'}
def get_final_voice(row):
    arr = [row['final_abstraction'] * abstraction_weight, row['final_grading_level'] * grading_level_weight, \
             row['final_reading_ease'] * reading_ease_weight, row['final_sgrank'] * sgrank_weight]
    idx = arr.index(max(arr))
    return row[cdict[idx]]
    
df['final_score'] = df.apply(lambda row: (row['final_abstraction'] + row['final_grading_level'] + row['final_reading_ease'] + row['final_sgrank'])/4 if row['final_abstraction'] >=0 else -1 , axis = 1)
df['final_voice'] = df.apply(get_final_voice, axis = 1)
df.final_score = df.final_score.apply(lambda x : round(x, 2))
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].sample(frac = 1).head(20)

,response,clauses_text_final,final_score,final_voice
501,I was curious as to the reason but didn't take...,"[I was curious as to the reason but, didn t ta...",0.45,P_bevb_x
444,"I'd listen, collaborate, communicate with my e...","[I d listen collaborate but, communicate with ...",0.46,A_def
225,"I am getting old, which is to say, I am an old...","[I am getting old, which is to say, I am an ol...",0.60,P_bevb_x
113,we started from within and worked on the notio...,"[we started from within and, worked on the not...",0.38,A_def
165,Is like sitting within the sacred heart of God...,"[Is like, sitting within the sacred heart of G...",0.41,A_def
464,"big, big existential questions, and little mic...",[big big existential questions and little micr...,0.47,Undefined
292,play soccer,[play soccer],0.44,A_def
403,I want to help them,[I want to help them],0.23,A_def
23,are expressions of one another.,[are expressions of one another],0.38,P_bevb_x
171,the anger is both a collective and personal ph...,[the anger is both a collective and personal p...,0.51,A_pron_x


In [9]:
suffix = "_weighted"  if WEIGHT_METRICS else ""
df[['response', 'clauses_text_final', 'final_score', 'final_voice']].to_csv("./output{}.csv".format(suffix), index = False)
df[['prompt', 'response', 'clauses_text_final', 'voice', 'abstraction_score_normalized', 'reading_ease_normalized', \
       'grading_level_normalized', 'sgrank_normalized',   \
    'final_abstraction', 'final_grading_level', 'final_reading_ease', 'final_sgrank', 'final_score', \
       'voice_abstraction', 'voice_grading_level', 'voice_reading_ease', 'voice_sgrank', 'final_voice']].to_csv("./debug{}.csv".format(suffix), index = False)

print("Written")

Written
